In [1]:
import pyam
import pandas as pd
import numpy as np

from pathlib import Path

<IPython.core.display.Javascript object>

Step 1: Read in the dataset.

In [2]:
df = pyam.IamDataFrame(
    Path(
        '../data/101_data.xlsx'
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../data/101_data.xlsx
pyam.core - INFO: Reading meta indicators


Step 2: Downselect the necessary data.

In [3]:
df_co2 = (
    df
    .filter(
        variable='*Emissions|CO2|Direct Only',
        region='World'
    )
)

In [4]:
df_co2.meta.groupby('Category')['Year of netzero CO2 emissions (Harm-Infilled) table'].describe()

,count,mean,std,min,25%,50%,75%,max
Category,,,,,,,,
C1,70.0,2052.685714,8.535591,2030.0,2048.0,2052.0,2055.75,2076.0
C2,106.0,2057.122642,6.142286,2044.0,2052.0,2058.0,2062.00,2069.0
C3,217.0,2071.788018,10.242488,2035.0,2066.0,2070.0,2077.00,2100.0


Step 3: Pull out the harmonised "original" net zero timings.

In [5]:
netzero_timings_original = df_co2.meta.loc[:, 'Year of netzero CO2 emissions (Harm-Infilled) table']

Step 4: Calculate the net zero timings for the reanalysed dataset.

In [6]:
df_co2_ts = (
    df_co2
    .timeseries()
)

In [7]:
def calculate_netzero(x):
    netzero = pyam.cross_threshold(x, 0.1)
    return netzero[0] if len(netzero) else np.nan

In [8]:
netzero_reanalysed = df_co2_ts.apply(
    lambda x: calculate_netzero(x),
    axis=1
)

In [9]:
netzero_reanalysed

model              scenario                 region  variable                                            unit     
AIM/CGE 2.0        SSP1-26                  World   AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  Gt CO2/yr    2075.0
                   SSP4-26                  World   AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  Gt CO2/yr    2078.0
AIM/CGE 2.1        CD-LINKS_NPi2020_1000    World   AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  Gt CO2/yr    2062.0
                   CD-LINKS_NPi2020_400     World   AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  Gt CO2/yr    2040.0
                   CO_Bridge                World   AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  Gt CO2/yr       NaN
                                                                                                                      ...  
WITCH-GLOBIOM 4.4  CD-LINKS_INDC2030i_1600  World   AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  Gt CO2/yr    2090.0
                  

In [10]:
(netzero_timings_original - netzero_reanalysed).describe()

count    370.000000
mean       1.859459
std        6.665181
min      -29.000000
25%       -1.000000
50%        1.000000
75%        5.000000
max       35.000000
dtype: float64